# Accompanying stats pipeline to help process ommatidial information

## To-do:
- Fix higher than 1 power correlation

In [1]:
from scipy import stats
import numpy as np
import os
import matplotlib.pyplot as plt
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.io as pio
pio.renderers.default='browser'
import h5py
import pickle
import json
from tqdm import notebook
%matplotlib qt
print(os.getcwd())
from scipy.spatial import ConvexHull

C:\Users\griff\Box\College\Honors\BERTT\DONE_HDFS


In [2]:
def t_test(base_f, d1_name, d2_name, to_compare, n_bins, eye_heights = []):
    if len(eye_heights)==0:
        eye_heights = [1,1]

    base_f = base_f
    dist1_name = d1_name
    dist2_name = d2_name

    to_compare = to_compare

    compare_f_name = 'all_'+to_compare+'.csv'

    loc1 = os.path.join(base_f, dist1_name, compare_f_name)
    loc2 = os.path.join(base_f, dist2_name, compare_f_name)

    dist1 = np.loadtxt(loc1)
    dist2 = np.loadtxt(loc2)
    
    if '33' in d1_name:
        real_height1 = eye_heights[0]*1200
    else:
        real_height1 = eye_heights[0]*600

    if '33' in d2_name:
        real_height2 = eye_heights[1]*1200
    else:
        real_height2 = eye_heights[1]*600
        
    if to_compare == 'dists':
        dist1 = dist1[dist1<100000]/real_height1
        dist2 = dist2[dist2<100000]/real_height2
    elif to_compare == 'angles' or to_compare=='rayleighs':
        dist1 = dist1[dist1<15]/real_height1
        dist2 = dist2[dist2<15]/real_height2
    elif to_compare == 'barlows':
        dist1 = dist1[dist1<7.5]/real_height1
        dist2 = dist2[dist2<7.5]/real_height2
    else:
        dist1 = dist1/real_height1
        dist2 = dist2/real_height2
    
    [_ , p_val] = stats.ttest_ind(dist1, dist2)

    fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)

    ax.hist(dist1, label=d1_name, alpha=.7, density=True, bins=n_bins)

    ax.hist(dist2, label=d2_name, alpha=.7, density=True, bins=n_bins)

    ax.legend(prop={'size': 10})
    ax.set_title(str('Normalized histogram of '+to_compare))
    
    
    return p_val

In [11]:
def multi_hist(base_f,d_names,to_compare,n_bins=100,d_categories=[], eye_heights = []):
    if len(eye_heights)==0:
        eye_heights = np.ones(len(d_names))*1500
    base_f = base_f
    to_compare = to_compare
    fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
    full_dist = []
    if len(d_categories)>0:
        if len(d_categories)==len(d_names):
            count = 0
            for d_name in d_names:
                compare_f_name = 'all_'+to_compare+'.csv'

                loc = os.path.join(base_f, d_name, compare_f_name)

                dist =  np.loadtxt(loc)
                print(loc)
                if '33' in loc:
                    real_height = eye_heights[count]*1200
                else:
                    real_height = eye_heights[count]*600
                
                multer = np.mean(eye_heights)
                
                if to_compare == 'dists':
                    dist = dist[dist<100000]/real_height
                    unit = '(nm/nm)'
                elif to_compare=='rayleighs':
                    dist = dist[dist<15]/real_height
                    unit = '(degrees/nm)'
                elif to_compare == 'angles':
                    dist = dist[dist<15]/real_height
                    unit = '(degrees/nm)'
                elif to_compare == 'barlows':
                    dist = dist[dist<7.5]
                    unit = '(no unit/nm)'
                else:
                    dist = dist/real_height
                    unit = '(no unit/nm)'

#                 counts, bins = np.histogram(dist/real_height, density=True, bins =n_bins)
#                 ax.hist(bins[:-1], bins, weights=counts, label=d_categories[count], alpha=.5)
#                 ax.hist(bins[:-1], counts, label=d_categories[count], alpha =.5)
                ax.hist(dist, density=True, bins = n_bins, label=d_categories[count], alpha=.5)
                print("Mean of "+d_categories[count]+" number "+ str(count)+": " + str(np.mean(dist)))
                count+=1
                full_dist.append(dist)
        else:
            print("Not the correct number of categories. Please give a category for every distribution.")
            return
    else:
        multi_hist(base_f, d_names, to_compare, d_categories=d_names)
       

    ax.legend(prop={'size': 10})
    ax.set_title(str('Density histogram of '+to_compare))
    ax.set_xlabel(to_compare+ ' scaled by eye height '+unit)
    ax.set_ylabel('Density')
    full_dist = np.hstack(full_dist)
    print(full_dist.shape)
    print(np.median(full_dist))
    print(np.std(full_dist))
    plt.show()

In [4]:
def two_scatter(base_f,d_names,to_compare,d_categories=[], power=1):
    base_f = base_f
    [comp1, comp2] = to_compare
    fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
    if len(d_categories)>0:
        if len(d_categories)==len(d_names):
            count = 0
            for d_name in d_names:
                comp1_f_name = 'all_'+comp1+'.csv'
                comp2_f_name = 'all_'+comp2+'.csv'
                loc1 = os.path.join(base_f, d_name, comp1_f_name)
                loc2 = os.path.join(base_f, d_name, comp2_f_name)
                distro1 =  np.loadtxt(loc1)
                distro2 =  np.loadtxt(loc2)
                mean1 = np.median(distro1)
                std1 = np.std(distro1)
                distro1_crop = distro1[distro1<mean1+std1]
                distro1_crop2 = distro1_crop[distro1_crop>mean1-std1]
                distro2_crop = distro2[distro1<mean1+std1]
                distro2_crop2 = distro2_crop[distro1_crop>mean1-std1]
                z = np.polyfit(distro1_crop2, distro2_crop2, power)
                p = np.poly1d(z)
                distro2_pred = p(distro1_crop2)
                plt.plot(distro1_crop2,
                         distro2_pred, '--')
                res = distro2_crop2-distro2_pred
                tot = distro2_crop2-np.mean(distro2_crop2)
                r_sq = 1 - np.dot(res,res) / np.dot(tot,tot)
                ax.text(5*(count+1),5*(count+1),d_categories[count]+ 'R^2 = '+str(r_sq))
                ax.scatter(distro1_crop2, distro2_crop2, label = d_categories[count])
                count+=1
        else:
            print("Not the correct number of categories. Please give a category for every distribution.")
            return
    else:
        for d_name in d_names:
                comp1_f_name = 'all_'+comp1+'.csv'
                comp2_f_name = 'all_'+comp2+'.csv'
                loc1 = os.path.join(base_f, d_name, comp1_f_name)
                loc2 = os.path.join(base_f, d_name, comp2_f_name)
                distro1 =  np.loadtxt(loc1)
                distro2 =  np.loadtxt(loc2)
                z = np.polyfit(distro1, distro2, power)
                p = np.poly1d(z)
                distro2_pred = p(distro1)
                plt.plot(np.sort(distro1), distro2_pred)
                res = distro2-distro2_pred
                tot = distro2-np.mean(distro2)
                r_sq = 1 - np.dot(res,res) / np.dot(tot,tot)
                ax.text(5*(count+1),5*(count+1),d_categories[count]+ 'R^2 = '+str(r_sq))
                ax.scatter(distro1, distro2, label = d_name)
                count+=1
    ax.set_xlabel(comp1)
    ax.set_ylabel(comp2)
    if comp1 == 'barlows':
        ax.set_xlim(0,10)
    if comp2 == 'angles':
        ax.set_ylim(0,30)
    ax.legend(prop={'size': 10})
    ax.set_title(str('Scatter of ' + comp1 + ' vs. ' + comp2))

In [5]:
def compare_cones(base_f, cone_locs):
    
    fig = go.Figure()
    base_f = base_f
    locs = [os.path.join(base_f,x,'platonic_cone.h5') for x in cone_locs]

    all_coords={}
    count = 0
    
    to_double = np.where([['33' in x for x in locs], ['32' in x for x in locs]])[1]
    
#     for i in locs:
#         f = h5py.File(i, 'r')
#         data = np.asarray(f['cone'])
#         f.close()
#         sparse_norm_coordall = [[0, 0, 0]]
#         test_coord = np.stack(np.where(data), axis=1)
#         norm_coord = test_coord - np.mean(test_coord, axis=0)
        
        
#         if np.any([str(count) in str(x) for x in to_double]):
#             sparse_norm_coord = norm_coord*2
#         else:
#             sparse_norm_coord = norm_coord
                
#         sparse_norm_coordall = np.vstack((sparse_norm_coordall,sparse_norm_coord))
#         all_coords[count] = sparse_norm_coordall

#         fig.add_trace(go.Scatter3d(
#         x=sparse_norm_coordall[1:, 0],
#         y=sparse_norm_coordall[1:, 1],
#         z=sparse_norm_coordall[1:, 2],
#         mode='markers',
#         opacity=0.05,
#         name = cone_locs[count],
#         marker = dict(
#                 size = 5,
#                 opacity = .1),
#         showlegend=True
#         ))
#         count +=1
#     maxer = np.max(list(all_coords.values())[0])
#     fig.update_layout(
#     scene = dict(
#         xaxis = dict(nticks=4, range=[-maxer, maxer],),
#                      yaxis = dict(nticks=4, range=[-maxer, maxer]),
#                      zaxis = dict(nticks=4, range=[-maxer, maxer],),
#     aspectmode='cube'),
#     width=700,
#     margin=dict(r=20, l=10, b=10, t=10))

#     fig.show()
    
    fig = go.Figure()
    count = 0
    for i in locs:
        f = h5py.File(i, 'r')
        data = np.asarray(f['cone'])
        f.close()
        sparse_norm_coordall = [[0, 0, 0]]
        test_coord = np.stack(np.where(data), axis=1)
        norm_coord = test_coord - np.mean(test_coord, axis=0)
        
        
        if np.any([str(count) in str(x) for x in to_double]):
            sparse_norm_coord = norm_coord*2
        else:
            sparse_norm_coord = norm_coord
        
        sparse_norm_coordall = sparse_norm_coord[ConvexHull(sparse_norm_coord).vertices]
        sparse_norm_coordall = np.vstack((sparse_norm_coordall,sparse_norm_coord))
        all_coords[count] = sparse_norm_coordall

        fig.add_trace(go.Mesh3d(
        x=sparse_norm_coordall[1:, 0],
        y=sparse_norm_coordall[1:, 1],
        z=sparse_norm_coordall[1:, 2],
        opacity=0.4,
        name = cone_locs[count],
        showlegend=True,
        alphahull=0
        ))
        count +=1
    maxer = np.max(list(all_coords.values())[0])
    fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[-maxer, maxer],),
                     yaxis = dict(nticks=4, range=[-maxer, maxer]),
                     zaxis = dict(nticks=4, range=[-maxer, maxer],),
    aspectmode='cube'),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))
    fig.show()
    

In [6]:
def t_test_volumes(base_f, pick_locs, n_bins=200):
    if np.size(pick_locs)>2:
        print('too many volumes')
        return
    base_f = base_f
    fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
    locs = [os.path.join(base_f,x+'_volumes.pickle') for x in pick_locs]
    both_dists = {}
    count = 0
    for loc in notebook.tqdm(locs):
        with open(loc, 'rb') as handle:
            vol_list = pickle.load(handle)
        datum = vol_list
        print(len(datum))
        if '33' in loc:
            datum = np.asarray(datum,dtype=np.uint8)*1200
        else:
            datum = np.asarray(datum,dtype=np.uint8)*600
        pc_dict = []
        distro = datum
        counts, bins = np.histogram(distro, density=True, bins = n_bins)
        ax.hist(bins[:-1], bins, weights=counts, alpha=.65, label=pick_locs[count])
        both_dists[loc]=distro
        count+=1
    ax.legend(prop={'size': 10})
    ax.set_title(str('Density histogram of volumes'))
    both_dists_arr = np.asarray(list(both_dists.values()))
    _, p_val = stats.ttest_ind(both_dists_arr[0], both_dists_arr[1])
    return p_val

In [7]:
def plot_volumes(base_f, pick_locs, n_bins=400):
    base_f = base_f
    fig, ax = plt.subplots(1, 1, sharey=True, tight_layout=True)
    locs = [os.path.join(base_f,x+'_volumes.pickle') for x in pick_locs]
    count = 0
    for loc in notebook.tqdm(locs):
        print("starting loop for " + str(loc))
        with open(loc, 'rb') as handle:
            vol_list = pickle.load(handle)
        print("loaded "+str(loc))
        datum = vol_list
        pc_dict = []
        print(len(datum))
        if '33' in loc:
            datum = np.asarray(datum, dtype=np.uint8)*1200
        else:
            datum = np.asarray(datum, dtype=np.uint8)*600
            
        distro = datum
        counts, bins = np.histogram(distro, density=True, bins = n_bins)
        ax.hist(bins[:-1], bins, weights=counts, alpha=.65, label = pick_locs[count])
        count +=1 
    ax.legend(prop={'size': 10})
    ax.set_title(str('Normalized histogram of volumes'))

In [13]:
base_f = './'
dist1='ygw05'
dist2='ygw15'
dist3='ygw33'
dist4='ygw34'
dist5='ygw06'
dist6='ygw18'

labels = ['C', 'C', 'N', 'N', 'D']
eye_heights_px = [2100, 2800, 980, 1050, 2430]#, 3150]

# t_test_volumes(base_f, [dist4,dist5])
multi_hist(base_f, [dist1,dist2,dist3,dist4,dist5], 'barlows', d_categories=labels, eye_heights=eye_heights_px)
# print(t_test(base_f, dist4, dist5, 'dists', 200, eye_heights=[eye_heights_px[2],eye_heights_px[3]]))
# compare_cones(base_f, [dist1,dist2,dist3,dist4,dist5])


./ygw05\all_barlows.csv
Mean of C number 0: 1.501202224977938
./ygw15\all_barlows.csv
Mean of C number 1: 2.3259528671735548
./ygw33\all_barlows.csv
Mean of N number 2: 1.1906159815635025
./ygw34\all_barlows.csv
Mean of N number 3: 0.9687257132724727
./ygw06\all_barlows.csv
Mean of D number 4: 1.618496273437063
(6401,)
1.2806475903648349
1.0683838499508864
